<a href="https://colab.research.google.com/github/grace3999/USV_Python/blob/colab/0_xr_create_annotations_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mount google drive containings required files: 1) csv of annotation features, 2) netcdf files
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import numpy as np
import pandas as pd
import os
import re

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

In [0]:
def get_dir_list(dir_path):
    """takes in a directory path which contains the selection tables of annotations created from Raven Soft Lite, 
    creates list of path names"""
    
    path_names = []
    dirs = os.listdir(dir_path)
    for file in dirs: 
        path_names.append(dir_path + "/" + file)

    return path_names   

In [0]:
#may need to be updated based on file naming scheme
def get_file_info(path_name, order):
    """takes in a file path for annotation selections table and finds the animal_number and session and saves each accordingly. 
    'order' refers to whether the animal or session is listed first in the path name
    each file should be named with animal number and exp (e.g. 100_CPA.Table.1.selections)"""
    
    if order == 'animal':
      animal_number, session = re.split('_|-', path_name.split('/')[-1].split('.')[0])[0:2]
    else:
      session, animal_number = re.split('_|-', path_name.split('/')[-1].split('.')[0])[0:2] 

    return animal_number, session

In [0]:
def create_animal_df(path_name, animal_number, session, ts_mult):
    """takes in a file path for annotation selections table, animal number, session, and timestamp multiple and creates data frame.
    computes closest timestamp (based on multiple) and adds animal number, session, and time stamp columns"""
    
    data = pd.read_table(path_name)
    data = pd.DataFrame(data = data)
    
    data['animal_number'] = [animal_number]*data.shape[0]
    data['session'] = [session]*data.shape[0]
    
    #Determine closest time stamp of each annotation and add as column to df
    data['Begin Time (s)_1000'] = data['Begin Time (s)']*1000
    time_lambda = lambda a: (a // ts_mult)*ts_mult
    data['time_stamp'] = time_lambda(data['Begin Time (s)_1000'])
    
    data = data[['animal_number', 'session', 'time_stamp', 'Annotation', 'High Freq (Hz)']]

    return data

In [0]:
def create_annot_df(dir_path, order, ts_mult, session_name, save_path):
    """takes in a directory path which contains the selection tables of annotations created from Raven Soft Lite,
    uses get_dir_list, get_file_info, and create_animal_df functions 
    to create a single data frame containing annotations for each file in dir_path"""
    
    annot_df = pd.DataFrame()
    
    path_names = get_dir_list(dir_path)
    
    for path in path_names:
        animal_number, session = get_file_info(path, order)
        animal_df = create_animal_df(path, animal_number, session, ts_mult)
        annot_df = annot_df.append(animal_df)
    
    #save as csv using name of dir_path
    annot_df.to_csv(str(save_path + '/annot_df_' + str(session_name) + '.csv'))
    
    return annot_df

In [0]:
path_cFos_CPA = '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/ravenlite_selection_tables/cFos/3x_CPA_pair_tables'
path_cFos_neutral = '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/ravenlite_selection_tables/cFos/3x_neutral_pair_tables'

path_PETr1_CPA = '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/ravenlite_selection_tables/PET_r1/CPAPair'

path_round2_CPA = '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/ravenlite_selection_tables/round2/CPApost'
path_round2_neutral = '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/ravenlite_selection_tables/round2/neutralpost'


In [30]:
order = 'animal'
ts_mult=22.5
session_name = 'r2_CPA'
save_path = '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/annotation_data_frames'

annot_df = create_annot_df(path_round2_CPA, order, ts_mult, session_name, save_path)
print(annot_df.shape)
annot_df.head()

(76, 5)


,animal_number,session,time_stamp,Annotation,High Freq (Hz)
0,625,CPApost,370080.0,bbc,61181.4
1,625,CPApost,548797.5,bbc,51532.8
0,624,CPApost,178245.0,low,27406.4
1,624,CPApost,262192.5,low multi,61831.6
2,624,CPApost,262305.0,low multi,62500.0
